In [1]:
import os
import sys

In [2]:
# os.chdir('/')
os.getcwd()

'/home/t/cproject/caption-generator'

# Have weights DO not delete

In [3]:
# os.system('pip uninstall funcyou')
# os.system('rm -rf /kaggle/working/caption-generator')
# # !ls /kaggle/working/caption-generator

In [4]:
os.getcwd()

'/home/t/cproject/caption-generator'

In [5]:
# !rm -rf 'caption-generator'
# os.chdir('/kaggle/working')
# os.system('git clone https://github.com/tikendraw/caption-generator.git -q')
os.chdir('caption-generator')

In [5]:
!pwd ; ls

/bin/bash: /home/t/miniconda3/envs/tf_new/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/home/t/cproject/caption-generator
caption-generator-for-kaggle.ipynb  local_config_cuda
caption-generator-kaggle-V2.ipynb   main.py
captiongenerator_notebook.ipynb     model.py
checkpoints			    preprocess_data.py
cleaned.csv			    preprocessing.py
config.py			    __pycache__
config.yaml			    test.py
data				    train.ipynb
embedding			    transformer_in_making.ipynb
Evaluate.txt			    transformer.png
funcyou				    transformer.py
get_data.py			    utils.py
input				    word_count.csv
libdevice.10.bc


In [7]:
if not os.path.exists('funcyou'):
    os.system('git clone https://github.com/tikendraw/funcyou.git')
os.system('pip install funcyou/. -q')
    
os.system('pip list | grep funcyou')

funcyou                                1.0.1


0

In [8]:
! pwd ; ls -h

/kaggle/working/caption-generator
Evaluate.txt			    logs
__pycache__			    main.py
caption-generator-for-kaggle.ipynb  model.png
captiongenerator_notebook.ipynb     model.py
checkpoints			    preprocess_data.py
cleaned.csv			    preprocessing.py
config.py			    saved_weights
config.yaml			    test.py
data				    train.ipynb
embedding			    transformer.png
funcyou				    transformer.py
get_data.py			    transformer_in_making.ipynb
libdevice.10.bc			    utils.py
log				    word_count.csv


In [9]:
# os.system('pip list | grep funcyou')

In [7]:
import numpy as np
# import pandas as pdd
import polars as pd
import random, math
import tensorflow as tf
import datetime
import sys
from functools import cache
from pathlib import Path
from tqdm import tqdm

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import (
    TextVectorization, Embedding, Dense, Attention, MultiHeadAttention, Flatten, Dropout,
    Concatenate, Activation, GlobalAveragePooling2D
    )
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.utils import array_to_img, img_to_array
import string
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, TensorBoard
from preprocessing import preprocess_text, mapper, clean_words, load_images_now

from config import config

from preprocess_data import clean_the_df
import polars as pl
from funcyou.utils import printt, dir_walk
import matplotlib.pyplot as plt
from collections import Counter
import regex as re
import yaml

from transformer import CaptionGenerator, Encoder, Decoder
from utils import create_model_checkpoint
from funcyou.plot import plot_history
import datetime

In [16]:
config_file_path = './config.yaml'

# Read the config file 
with open(config_file_path, 'r') as file:
    config = yaml.safe_load(file)


IMAGE_DIR                               = Path(config['image_dir'])
IMG_SIZE                                = config['img_size']
CHANNELS                                = config['channels']
IMG_SHAPE                               = config['img_shape']
MAX_LEN                                 = config['max_len']
BATCH_SIZE                              = config['batch_size']
EPOCHS                                  = config['epochs']
LEARNING_RATE                           = config['learning_rate']
UNITS                                   = config['units']
TEST_SIZE                               = config['test_size']
VALIDATION_SIZE                         = config['val_size']
EMBEDDING_DIMENSION                     = config['embedding_dimension']
GLOVE_PATH                              = config['glove_path']
D_MODEL                                 = config['d_model']
NUM_HEADS                               = config['num_heads']    
NUM_LAYERS                              = config['num_layers']    
DFF                                     = config['dff']
PATCH_SIZE                              = config['patch_size']    
TRANSFORMER_LAYERS                      = config['transformer_layers']        

NUM_PATCHES = (IMG_SIZE // PATCH_SIZE) ** 2 


In [17]:
START_TOKEN = 'startseq'
END_TOKEN = 'endseq'

In [13]:
# CAPTION_FILE = Path('/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv')
# IMAGE_DIR = Path('/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images/flickr30k_images')

In [15]:
IMG_SIZE = 224
BATCH_SIZE = 32
IMG_SHAPE = (IMG_SIZE,IMG_SIZE,3)
D_MODEL = 256
MAX_LEN = 50
DFF = 1024
PATCH_SIZE = 32
NUM_PATCHES = (IMG_SIZE // PATCH_SIZE) ** 2
NUM_PATCHES

49

# Curating Dataframe

1. Null/Bad Entries removed
2. Columns names stripped and lowered (further cleaning will be done while tokenizing)
3. Low frequency word (<5) has been removed 
4. Added start and end tokens

we are keeping words in comments which atleast has occured  5 times( there are unnecessary word which we don't want) 
there are ~12000 words that doesn't even gets repeated 5 times in 150000 lines

In [16]:
# Cleaning it 
# clean_the_df(CAPTION_FILE, '.')

In [18]:
df = pd.read_csv('cleaned.csv')

try:
    df.drop('Unnamed: 0',axis=1, inplace=True)
except:
    pass

df.head()

image_name,comment_number,comment,no_rare_words,sent_len
str,i64,str,bool,i64
"""1000092795.jpg…",0,"""startseq Two …",true,18
"""1000092795.jpg…",1,"""startseq Two …",true,11
"""1000092795.jpg…",2,"""startseq Two …",true,12
"""1000092795.jpg…",3,"""startseq A ma…",true,12
"""1000092795.jpg…",4,"""startseq Two …",true,7


In [19]:
df = df.with_columns([
    pl.col("image_name").apply(lambda x: str(IMAGE_DIR / x) ).alias('image_path') 
])

# this code it to check images in dataframe are actual files in the directory (commenting it out: takes 2 minutes)
# df = df.with_columns([
#     pl.col("image_path").apply(lambda x: os.path.isfile(x)).alias('img_exists') 
# ])

df.head()


image_name,comment_number,comment,no_rare_words,sent_len,image_path
str,i64,str,bool,i64,str
"""1000092795.jpg…",0,"""startseq Two …",true,18,"""input/flickr30…"
"""1000092795.jpg…",1,"""startseq Two …",true,11,"""input/flickr30…"
"""1000092795.jpg…",2,"""startseq Two …",true,12,"""input/flickr30…"
"""1000092795.jpg…",3,"""startseq A ma…",true,12,"""input/flickr30…"
"""1000092795.jpg…",4,"""startseq Two …",true,7,"""input/flickr30…"


In [20]:
# df['img_exists'].value_counts(), df.shape

In [21]:
print(df.shape)
print(df.to_pandas().info())
print(df.columns)

(158914, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158914 entries, 0 to 158913
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   image_name      158914 non-null  object
 1   comment_number  158914 non-null  int64 
 2   comment         158914 non-null  object
 3   no_rare_words   158914 non-null  bool  
 4   sent_len        158914 non-null  int64 
 5   image_path      158914 non-null  object
dtypes: bool(1), int64(2), object(3)
memory usage: 6.2+ MB
None
['image_name', 'comment_number', 'comment', 'no_rare_words', 'sent_len', 'image_path']


### **For the Sake of COmputation power and time we will train with one caption per image rather than 5 caption per image**

Finding median sentence length for all class of comment numbers, to ensure that our MAX_LEN covers all of them

In [22]:

# Group by a categorical column
# grouped = df.select_dtypes(include=['float64', 'int64']).groupby('comment_number')
grouped = df.groupby('comment_number')

# Apply aggregation functions on numerical columns
aggregated = grouped.agg(
    pl.col('sent_len').mean().alias('mean'), 
    pl.col('sent_len').median().alias('median'), 
    pl.col('sent_len').min().alias('min'), 
    pl.col('sent_len').max().alias('max'), 
    pl.col('sent_len').count().alias('count')
)
aggregated

comment_number,mean,median,min,max,count
i64,f64,f64,i64,i64,u32
0,19.871315,19.0,6,79,31783
1,15.859327,15.0,6,41,31783
2,13.602869,13.0,4,34,31783
3,11.771639,11.0,4,36,31783
4,9.871437,10.0,3,38,31782


In [23]:
# i randomly chose 2nd comment of all picture you can chose anything bw 0 to 4
df = df.filter(
     (pl.col("sent_len") <= MAX_LEN) #& (pl.col("comment_number") == 1)  #& (pl.col("img_exists") == True) 
)
print(df.shape)
df.head()

(158855, 6)


image_name,comment_number,comment,no_rare_words,sent_len,image_path
str,i64,str,bool,i64,str
"""1000092795.jpg…",0,"""startseq Two …",true,18,"""input/flickr30…"
"""1000092795.jpg…",1,"""startseq Two …",true,11,"""input/flickr30…"
"""1000092795.jpg…",2,"""startseq Two …",true,12,"""input/flickr30…"
"""1000092795.jpg…",3,"""startseq A ma…",true,12,"""input/flickr30…"
"""1000092795.jpg…",4,"""startseq Two …",true,7,"""input/flickr30…"


now we are just using 1/5 dataset

# Tokenize

In [24]:
def preprocess_text(text):

    text = tf.strings.lower(text)

    text = tf.strings.regex_replace(text, r'\d', '')

    # Remove any punctuations
    text = tf.strings.regex_replace(text, '[%s]' % re.escape(
        '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'), '')

    # Remove single characters
    text = tf.strings.regex_replace(text, r'\b\w\b', '')
    # Keep space, a to z, and select punctuation.
    text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
    # Add spaces around punctuation.
    text = tf.strings.regex_replace(text, '[.?!,¿|]', r' \0 ')
    # Strip whitespace.
    text = tf.strings.strip(text)

    return text


In [25]:
dff = df['comment'].to_list()
print(dff[:10], end='\n')
del(dff)

['startseq  Two young guys with shaggy hair look at their hands while hanging out in the yard  endseq', 'startseq  Two young White males are outside near many bushes  endseq', 'startseq  Two men in green shirts are standing in a yard  endseq', 'startseq  A man in a blue shirt standing in a garden  endseq', 'startseq  Two friends enjoy time together  endseq', 'startseq  Several men in hard hats are operating a giant pulley system  endseq', 'startseq  Workers look down from up above on a piece of equipment  endseq', 'startseq  Two men working on a machine wearing hard hats  endseq', 'startseq  Four men on top of a tall structure  endseq', 'startseq  Three men on a large rig  endseq']


In [26]:

#tokenizer
tokenizer = TextVectorization(standardize=preprocess_text)
tokenizer.adapt(df['comment'].to_list())


word_to_id = tf.keras.layers.StringLookup(vocabulary=tokenizer.get_vocabulary(), mask_token='', oov_token='[UNK]')
id_to_word = tf.keras.layers.StringLookup(vocabulary=tokenizer.get_vocabulary(), mask_token='', oov_token='[UNK]', invert=True)


# making Dataset

In [58]:

def mapper(x, y, tokenizer):
    x = load_images_now(x)
    y = tokenizer(y)

    y_in = y[:-1]
    y_in =  tf.pad(y_in, [[0, MAX_LEN - tf.shape(y_in)[0]]] , constant_values=0)

    y_out = y[1:]
    y_out =  tf.pad(y_out, [[0, MAX_LEN - tf.shape(y_out)[0]]], constant_values=0)

    return (x, y_in), y_out

# load image model
resnet = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_tensor=tf.keras.layers.Input(shape=IMG_SHAPE))

@tf.function
def load_images_now(x):
    image_data = tf.io.read_file(x)
    image_features = tf.image.decode_jpeg(image_data, channels=CHANNELS)
    image_features = tf.image.resize_with_pad(
        image_features, target_height=IMG_SIZE, target_width=IMG_SIZE)


    return image_features


In [59]:
# import matplotlib.pyplot as plt
# ss = '/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images/flickr30k_images/631949256.jpg'

# im = plt.imread(ss)
# plt.imshow(im)

In [60]:
# random_image_filepath = random.choices(df['image_path'].to_list(), k=1)[0]
# p_image = load_images_now(str(random_image_filepath))
# array_to_img(p_image)


In [61]:
# Creating dataset
TEST_SIZE = config['test_size']
VAL_SIZE =  config['val_size']

train, val = train_test_split(
    df[['image_path', 'comment']],  test_size=VAL_SIZE, random_state=11)
train, test = train_test_split(
    train[['image_path', 'comment']],  test_size=TEST_SIZE, random_state=11)


train_data = tf.data.Dataset.from_tensor_slices((train['image_path'].to_list(), train['comment'].to_list()))
test_data = tf.data.Dataset.from_tensor_slices((test['image_path'].to_list(), test['comment'].to_list()))
val_data = tf.data.Dataset.from_tensor_slices((val['image_path'].to_list(), val['comment'].to_list()))


train_data = train_data.map(lambda x,y:mapper(x, y, tokenizer)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data =   test_data.map(lambda x,y:mapper(x, y, tokenizer)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_data =     val_data.map(lambda x,y:mapper(x, y, tokenizer)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# resnet_output_flattened_shape = 8*8*2048

print("Number of training samples: %d" %
      tf.data.experimental.cardinality(train_data))
print("Number of validation samples: %d" %
      tf.data.experimental.cardinality(val_data))
print("Number of test samples: %d" %
      tf.data.experimental.cardinality(test_data))

VOCAB_SIZE = tokenizer.vocabulary_size()
print("Vocabulary size: %d" % VOCAB_SIZE)


Number of training samples: 17921
Number of validation samples: 993
Number of test samples: 944
Vocabulary size: 7513


2023-05-24 11:24:41.247662: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'args_0' with dtype string
	 [[{{node args_0}}]]
2023-05-24 11:24:41.316523: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'args_0' with dtype string
	 [[{{node args_0}}]]
2023-05-24 11:24:41.379873: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'args_0' with dtype string
	 [[{{node args_0}}]]


In [62]:
train.shape, len(train_data)*BATCH_SIZE

((143366, 2), 143368)

In [63]:
for (img_in, txt_in), txt_out in train_data.take(1):
    # print(f'{i.numpy().decode():<40} {j.numpy()}')
    print('x     : ', img_in.shape)
    print('y_in.shape(one batch)  : ', txt_in.shape)
    
    print('y_in  : ', txt_in[0])
    
    print('y_out.shape(one batch)  : ', txt_out.shape)
    print('y_out : ', txt_out[0])
    print('\n')


x     :  (8, 256, 256, 3)
y_in.shape(one batch)  :  (8, 50)
y_in  :  tf.Tensor(
[   2  398 2201    4  403   20  936    6    5  209    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0], shape=(50,), dtype=int64)
y_out.shape(one batch)  :  (8, 50)
y_out :  tf.Tensor(
[ 398 2201    4  403   20  936    6    5  209    3    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0], shape=(50,), dtype=int64)




2023-05-24 11:24:44.770566: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [143366]
	 [[{{node Placeholder/_0}}]]


# Model

In [33]:
enc = Encoder( NUM_LAYERS, D_MODEL, NUM_HEADS, DFF, PATCH_SIZE, NUM_PATCHES, dropout_rate=0.1)
dec = Decoder( NUM_LAYERS, D_MODEL, NUM_HEADS, DFF, VOCAB_SIZE, dropout_rate=0.1)
model = CaptionGenerator(NUM_LAYERS, D_MODEL, NUM_HEADS, DFF, VOCAB_SIZE, PATCH_SIZE, NUM_PATCHES, dropout_rate=0.2)

## shape check

In [34]:
img_in.shape, txt_in.shape

(TensorShape([32, 224, 224, 3]), TensorShape([32, 50]))

In [35]:
ee = enc(img_in)
ee.shape

TensorShape([32, 49, 256])

In [36]:
dd = dec(x = txt_in, context = ee)
dd.shape

TensorShape([32, 50, 256])

In [37]:
output = model((img_in, txt_in))
output.shape

TensorShape([32, 50, 7513])

In [38]:

model

### Custom accuracy and loss

In [39]:

def masked_loss(y_true, y_pred):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        reduction='none')
    loss = loss_fn(y_true, y_pred)

    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

def masked_accuracy(y_true, y_pred):
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)

    match = y_true == y_pred

    mask = y_true != 0

    match = match & mask

    match = tf.cast(match, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)

    return tf.reduce_sum(match)/tf.reduce_sum(mask)


In [40]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=300):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [41]:
# learning_rate = CustomSchedule(D_MODEL)
learning_rate = 0.01
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [42]:
# plt.plot(learning_rate(tf.range(1000, dtype=tf.float32)))
# plt.ylabel('Learning Rate')
# plt.xlabel('Train Step')

In [43]:
model.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])



In [44]:
!ls 'saved_weights'

checkpoint  trial.data-00000-of-00001  trial.index


In [45]:
# model.load_weights('saved_weights/trial')

In [46]:
os.makedirs('log', exist_ok=True)
csv_logger = CSVLogger('./log/training.log',append=True )
tb_callback = tf.keras.callbacks.TensorBoard('./logs', update_freq=1)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [47]:
EPOCHS = 300
print(len(train_data) // EPOCHS, len(val_data) // EPOCHS)

# steps_per_epoch = int(1*(len(train_data) / EPOCHS))
# validation_steps =  int(1*(len(val_data) / EPOCHS))
# print(steps_per_epoch, validation_steps)

steps_per_epoch = 300
validation_steps = 2
print(steps_per_epoch, validation_steps)


14 0
300 2


In [48]:
len(train_data), len(val_data), len(test_data)

(4481, 249, 236)

In [49]:
print(VOCAB_SIZE)
{"expected_loss": np.log(VOCAB_SIZE),
 "expected_acc": 1/VOCAB_SIZE}

7513


{'expected_loss': 8.92439013236916, 'expected_acc': 0.0001331026221216558}

In [50]:
def sizeof_fmt(num, suffix="B"):
    """by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified"""
    for unit in ["", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"]:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, "Yi", suffix)


for name, size in sorted(
    ((name, sys.getsizeof(value)) for name, value in locals().items()),
    key=lambda x: -x[1],
)[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                   TensorBoard:  2.0 KiB
                          _i30:  1.4 KiB
                          _i11:  1.3 KiB
                          _i24:  1.3 KiB
                          _i10:  1.2 KiB
                     CSVLogger:  1.2 KiB
                        config:  1.1 KiB
                          tqdm:  1.0 KiB
                     Attention:  1.0 KiB
                       Flatten:  1.0 KiB


## model.fit

In [51]:
history = model.fit(train_data.repeat(),
                    epochs=EPOCHS,
                    validation_data=val_data,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    callbacks=[
                        # decay_callback,
                        csv_logger,
                        create_model_checkpoint(model_name = 'capgen', save_dir = 'checkpoints', monitor = 'masked_acc'),
                        tensorboard_callback
                                ]
                    )

Epoch 1/300
300/300 [==============================] - 103s 236ms/step - loss: 12.6698 - masked_accuracy: 0.0105 - val_loss: 12.8234 - val_masked_accuracy: 0.0038
Epoch 2/300
300/300 [==============================] - 97s 324ms/step - loss: 12.7218 - masked_accuracy: 0.0116 - val_loss: 12.8234 - val_masked_accuracy: 0.0038
Epoch 3/300
300/300 [==============================] - 82s 275ms/step - loss: 12.7140 - masked_accuracy: 0.0120 - val_loss: 12.8234 - val_masked_accuracy: 0.0038
Epoch 4/300
300/300 [==============================] - 75s 249ms/step - loss: 12.7091 - masked_accuracy: 0.0122 - val_loss: 12.8234 - val_masked_accuracy: 0.0038
Epoch 5/300
300/300 [==============================] - 71s 236ms/step - loss: 12.7155 - masked_accuracy: 0.0138 - val_loss: 13.4092 - val_masked_accuracy: 0.0194
Epoch 6/300
300/300 [==============================] - 62s 208ms/step - loss: 13.6083 - masked_accuracy: 0.0266 - val_loss: 14.1433 - val_masked_accuracy: 0.0397
Epoch 7/300
300/300 [======

KeyboardInterrupt: 

In [ ]:
# plot_history(history,plot = ['loss','masked_accuracy'], split = ['train','val'], )

In [ ]:
model.save_weights('saved_weights/trial1')

In [ ]:
# %tensorboard --logdir logs/gradient_tape


# Inference

In [ ]:
from tensorflow.keras.utils import load_img

In [ ]:
img = plt.imread(random_image_filepath)
print(img.shape)
plt.imshow(img)

In [ ]:
for (imga, txin), txout in test_data.take(1):
    print(imga.shape)
    print(txin.shape)
    print(txout.shape)
    break

In [ ]:
class CaptionGeneratorMain(tf.Module):
    def __init__(self, tokenizer, transformer):
        self.tokenizer = tokenizer
        self.transformer = transformer
    
    def preprocessing(self, img, img_size = IMG_SIZE):
        return tf.image.resize_with_pad(img, target_height=img_size, target_width=img_size)
    
    def clean_output(self, output_ids):
        caption_array = id_to_word(output_ids).numpy()[0]
        array_of_strings = [s.decode("utf-8") for s in caption_array]
        caption = tf.strings.join(array_of_strings, ' ')
        caption_clean = tf.strings.regex_replace(caption, START_TOKEN, ' ')
        caption_clean = tf.strings.regex_replace(caption_clean, END_TOKEN, ' ')
        return caption_clean
        
        
        
    def __call__(self, img, max_len=MAX_LEN, k = 3):
        assert isinstance(img, tf.Tensor)
        if img.ndim == 3:
            img = tf.expand_dims(img, 0)
        
        batch_size = img.shape[0]
#         print('batch_size: ',batch_size)

        img = self.preprocessing(img)

        start_id = self.tokenizer(START_TOKEN)
        end_id = self.tokenizer(END_TOKEN)

        caption_ids = tf.fill((batch_size,1), start_id)
        print('caption_ids: ',caption_ids.shape)
        
        beam_score = tf.fill((k,1), 1.0)
        print('beam_score: ',beam_score.shape)
        
        new_cap = list()
        
        for h, caption in enumerate(caption_ids):
            for i in tf.range(max_len):
                predictions = self.transformer([img, tf.expand_dims(caption, 0)], training=False)
    #             print('output shape: ',predictions.shape)

                k_words_tokens = tf.squeeze(tf.argsort(predictions, direction='DESCENDING')[:,:,:k])
                print('k_words_tokens: ',k_words_tokens)

                pred_prob = tf.gather(predictions[0,0,:], k_words_tokens)
                print('pred_prob: ',pred_prob)

                
                
                
                
                
                
                
                
                
                
                
#                 if i ==0:
#                     beam_score  = tf.reshape(pred_prob, (k,1))
#                     k_words_tokens = tf.reshape(tf.cast(k_words_tokens, caption_ids.dtype),(k,1))
#                     caption_ids = tf.broadcast_to(caption_ids,(k,1))
#                     caption_ids = tf.concat([caption_ids ,k_words_tokens], axis = -1)
#                 else:
#                     beam_score = beam_score[i]- tf.math.log(pred_prob)
                    



                print(beam_score)

                print(caption_ids)
                break
        

In [ ]:
hey = CaptionGeneratorMain(tokenizer, model)
img = tf.constant(img)
ss= hey(img)

In [ ]:
class CaptionGeneratorMain(tf.Module):
    def __init__(self, tokenizer, transformer):
        self.tokenizer = tokenizer
        self.transformer = transformer
        
    def __call__(self, img, max_len=MAX_LEN, k = 5):
        assert isinstance(img, tf.Tensor)
        if img.ndim == 3:
            img = tf.expand_dims(img, 0)
        
        batch_size = img.shape[0]
#         print('batch_size: ',batch_size)

        img = self.preprocessing(img)

        start_id = self.tokenizer(START_TOKEN)
        end_id = self.tokenizer(END_TOKEN)

        caption_ids = tf.fill((batch_size,1), start_id)
        print('caption_ids: ',caption_ids.shape)
        
        beam_prob = tf.fill((k,1), 1.0)
        

        for i in tf.range(max_len):
#             output = tf.transpose(output_array.stack())
#             print('start shape :',output.shape)
#             print('txt in',output)
            predictions = self.transformer([img, caption_ids], training=False)
#             print('input: ',id_to_word(output))
            print('output shape: ',predictions.shape)

#             print('output: ',id_to_word(tf.argmax(predictions, -1)))
#             print(' ')
            # Select the last token from the `seq_len` dimension.
    
            
            k_words_tokens = list(tf.argsort(predictions, direction='DESCENDING')[:,:,:k])
            pred_prob = tf.gather(predictions[0,0,:], k_words_tokens)

            
            pred_prob = tf.reshape(pred_prob, (k,1))
            print('pred_prob: ',pred_prob.shape)
            
#             log_pred_prob = tf.math.log(pred_prob)
            print('before:' ,beam_prob.shape)
            print('beam_prob: ',beam_prob)
            print('pred_prob: ',pred_prob)
            if i == 0:
                beam_prob = tf.concat([beam_prob, pred_prob], axis =1)
            
            print('beam_prob: ',beam_prob)
            print('after:' ,beam_prob.shape)
# #             printt(log_pred_prob, pred_prob, beam_prob, log_beam_prob)
#             find = tf.matmul(log_beam_prob, log_pred_prob)
#             print(find)
#             selected_ids = tf.argmin(find, axis = 0)
#             print('selected_ids: ',selected_ids)

#             print('k_words_tokens: ',k_words_tokens)
# #             print('k_words_tokens shape: ', k_words_tokens.shape)
#             print('to words: ',id_to_word(k_words_tokens))
#             print(pred_prob)
#             print('pred prob shape: ', pred_prob.shape)
            

# #             predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`

#             predictions  = tf.reshape(k_words_tokens, (k,1))
#             print('predictions beam: ',predictions.shape)
    
        
#             aa = tf.broadcast_to(caption_ids,(batch_size*k,1))
#             print('aa shape:',aa.shape)
#             caption_ids = tf.concat([aa, tf.cast(predictions, caption_ids.dtype)], axis = 1)
            
#             print('concat: ',caption_ids)
#             print('beam shape: ',caption_ids.shape)
#             print('-'*20)
            
# #             predicted_id = tf.argmax(predictions, axis=-1)

# #             # Concatenate the `predicted_id` to the output which is given to the
# #             # decoder as its input.
# # #             output_array = output_array.write(i+1, predicted_id[0])
# #             caption_ids= tf.concat([caption_ids, predicted_id], axis=-1)
# #             print(caption_ids) 

            
# # #             print(output_array.stack())
# #             if predicted_id == end_id:
# #                 break
            break
# #         output = tf.transpose(output_array.stack())
# #         caption_ids
        
# #         print(caption)

#         return caption_ids #, self.clean_output(caption_ids)
# #       

    def preprocessing(self, img, img_size = IMG_SIZE):
        return tf.image.resize_with_pad(img, target_height=img_size, target_width=img_size)
    
    def clean_output(self, output_ids):
        caption_array = id_to_word(output_ids).numpy()[0]
        array_of_strings = [s.decode("utf-8") for s in caption_array]
        caption = tf.strings.join(array_of_strings, ' ')
        caption_clean = tf.strings.regex_replace(caption, START_TOKEN, ' ')
        caption_clean = tf.strings.regex_replace(caption_clean, END_TOKEN, ' ')
        return caption_clean
        
        

        

In [ ]:
ss

In [ ]:
array_to_img(img)

In [ ]:
@tf.function
def load_images_now(x):
    image_data = tf.io.read_file(x)
    image_features = tf.image.decode_jpeg(image_data, channels=CHANNELS)
    image_features = tf.image.resize_with_pad(
        image_features, target_height=IMG_SIZE, target_width=IMG_SIZE)
    return image_features

## Saving the Model

In [ ]:
# model.save(f'saved_model/best-{datetime.datetime.now()}-{EPOCHS}.h5')
# model.save_weights(f'model_weights/best_so_far_final/')

In [ ]:
!ls saved_model

In [ ]:
!ls -la -h 'saved_model/'

# Prediction

In [ ]:
pred = model.predict(test_data.take(1))
print(pred.shape)

In [ ]:
ppred = tf.argmax(pred, axis = -1)
print(ppred.shape)

In [ ]:
for i in ppred:
    print(tf.strings.join(id_to_word(i), ' ').numpy())
    print('')